In [ ]:
import cv2
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import os
import random
import gc
import warnings

warnings.filterwarnings('ignore')

In [ ]:
dir_treino = 'treino'
dir_teste = 'teste'

treino_demonstracoes = ['treino/{}'.format(i) for i in os.listdir(dir_treino) if 'positivo' in i]
treino_ndemonstracoes = ['treino/{}'.format(i) for i in os.listdir(dir_treino) if 'negativo' in i]

imgs_teste = ['teste/{}'.format(i) for i in os.listdir(dir_teste) if 'teste' in i]

imgs_treino = treino_demonstracoes + treino_ndemonstracoes
# une os datasets (se tiver quantidade suficiente repartir com [:xvalor])
random.shuffle(imgs_treino)
# embaralha as imagens

del treino_demonstracoes
del treino_ndemonstracoes
gc.collect()
# limpar o que n vai mais ser usado e coleta lixo para salvar memoria

In [ ]:
nlinhas = 150
ncolunas = 150
canais = 3
# declaração das dimensões das imagens, 3 canais para RGB e 1 para Grayscale

# função para ler e processar as imagens para um formato aceitavel para o modelo, retorna 2 arrays, um de imagens redimensionadas e um de rotulos
def ler_e_processar_imagens(lista_de_imagens):
    X = [] #imagens
    y = [] #rotulos

    for imagem in lista_de_imagens:
        X.append(cv2.resize(cv2.imread(imagem, cv2.IMREAD_COLOR), (nlinhas, ncolunas), interpolation=cv2.INTER_CUBIC))
        # coleta os rotulos
        if 'positivo' in imagem:
            y.append(1)
        if 'negativo' in imagem:
            y.append(0)
    
    return X, y

In [ ]:
# processa as imagens de treino e seus rotulos
X, y = ler_e_processar_imagens(imgs_treino)

In [ ]:
import seaborn as sns
del imgs_treino
gc.collect()

X = np.array(X)
y = np.array(y)
# conversão das listas para arrays numpy

sns.countplot(y)
plt.title('Rotulos')
# plota os rotulos para conferirmos se esta correto

In [ ]:
print("O formato das imagens de treino é: ", X.shape)
print("O formato dos rotulos é: ", y.shape)

In [ ]:
# divide o dataset em imagens de treino e teste
from sklearn.model_selection import train_test_split
X_treino, X_val, y_treino, y_val = train_test_split(X, y, test_size=0.20, random_state=2)

print("Formato das imagens de treino é: ", X_treino.shape)
print("Formato das imagens de validação: ", X_val.shape)
print("Formato dos rotulos é: ", y_treino.shape)
print("Formato dos rotulos é: ", y_val.shape)

In [ ]:
del X
del y
gc.collect()
# limpeza de memoria

ntreino = len(X_treino)
nval = len(X_val)
# pega o tamanho dos dados de treino e validação

batch_size = 32
# inicialmente usar batch_size de 32, o batch_size deve ser fator de 2

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights = 'imagenet', include_top = False, input_shape = (150,150,3))
# chama o modelo pretreinado, com os pesos da imagenet e sem a camada do cima, para adicionar a que é precisa

conv_base.summary()
# sumario das camada de convolução

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

modelo = models.Sequential()
modelo.add(conv_base)
modelo.add(layers.Flatten())
modelo.add(layers.Dense(256, activation='relu'))
modelo.add(layers.Dense(1, activation='sigmoid'))
# importado camadas e modelos do keras e combinados a camada de convolução pegada anteriormente

modelo.summary()
# mostra o modelo

In [ ]:
print("Número de pesos treinaveis antes de congelar a conv_base: ", len(modelo.trainable_weights))
conv_base.trainable = False
print("Número de pesos treinaveis depois de congelar a conv_base: ", len(modelo.trainable_weights))

In [ ]:
from tensorflow.keras import optimizers

modelo.compile(loss = "binary_crossentropy", optimizer = optimizers.RMSprop(lr = 2e-5), metrics = ["acc"])
# compila o modelo usando o otimizador RMS prop com um learning rate de 2e-5, e loss com binary_crossentropy já que é uma classificação binaria

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# aumentação já que o dataset é pequeno
datagen_treino = ImageDataGenerator(rescale=1./255,   
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

# dados de validação não precisam de aumentação, apenas de re-escala
datagen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
gerador_treino = datagen_treino.flow(X_treino, y_treino, batch_size = batch_size)
gerador_val = datagen_val.flow(X_val, y_val, batch_size = batch_size)
# criação dos geradores de imagem

In [ ]:
historico = modelo.fit_generator(gerador_treino,
                              steps_per_epoch = ntreino // batch_size,
                              epochs = 64,
                              validation_data = gerador_val,
                              validation_steps = nval // batch_size)

# treino com 64 epochs

In [ ]:
modelo.save_weights("pesos_modelo_inceptionresnetv2_teste_150_64.h5")
modelo.save("modelo_inceptionresnetv2_teste_150_64.h5")
# salvamento dos pesos e do modelo

In [ ]:
# pega os detalhes do historico
acc = historico.history['acc']
val_acc = historico.history['val_acc']
loss = historico.history['loss']
val_loss = historico.history['val_loss']

epochs = range(1, len(acc) + 1)

# precisão de treino e validação
plt.plot(epochs, acc, 'b', label='Precisão treino')
plt.plot(epochs, val_acc, 'r', label='Precisão validação')
plt.title('Precisão de treino e validação')
plt.legend()

plt.figure()
# perda de treino e validação
plt.plot(epochs, loss, 'b', label='Perda de treino')
plt.plot(epochs, val_loss, 'r', label='Perda de validação')
plt.title('Perda de treino e validação')
plt.legend()

plt.show()
# mostra os dados

In [ ]:
def suaviza(pontos, fator=0.7):
    suaviza_pts = []
    for ponto in pontos:
        if suaviza_pts:
            anterior = suaviza_pts[-1]
            suaviza_pts.append(anterior * fator + ponto * (1 - fator))
        else:
            suaviza_pts.append(ponto)
    return suaviza_pts

# função para suavizar os graficos

In [ ]:
plt.plot(epochs, suaviza(acc), 'b', label='Precisão de treino')
plt.plot(epochs, suaviza(val_acc), 'r', label='Precisão de validação')
plt.title('Precisão de treino e validação')
plt.legend()
plt.show()
# mostra os graficos suavizados

In [ ]:
X_teste, y_teste = ler_e_processar_imagens(imgs_teste[0:10])
x = np.array(X_teste)
datagen_teste = ImageDataGenerator(rescale = 1./255)
# tenta prever as 10 primeiras imagens do dataset de teste

In [ ]:
i = 0
colunas = 5
rotulos_texto = []
plt.figure(figsize=(30,20))
for batch in datagen_teste.flow(x, batch_size=1):
    pred = modelo.predict(batch)
    if pred > 0.5:
        rotulos_texto.append('positivo')
    else:
        rotulos_texto.append('negativo')
    plt.subplot(int(5 / colunas + 1), colunas, i + 1)
    plt.title('Result ' + rotulos_texto[i])
    imgplot = plt.imshow(batch[0])
    i += 1
    if i % 10 == 0:
        break
plt.show()